<a href="https://colab.research.google.com/github/zelalemamera-stonybrook/projects-sandbox/blob/main/Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import re

In [2]:
with open('/content/drive/MyDrive/charles_dickens_christmas_carol.txt', 'r') as f:
  scroodge_book = f.read()

In [3]:
def clean_sentences(sentence_list):
  '''
  Add SOS - start of sentence and EOS end of sentence markers.
  Add EOSq for end of sentence question
  replace ; : with EOS
  replace ! with ,
  remove - i.e. treat every compound word as one unit
  remove all numbers
  remove \
  remove "
  '''
  cleaned_sentence_list = []
  for sentence in sentence_list:
    filtered = re.sub(r"[;:,]", r"", sentence)
    filtered = re.sub(r"\.", r" eos", filtered)
    filtered = re.sub(r"\?", r" eosq", filtered)
    filtered = re.sub(r"!", r"", filtered)
    filtered = re.sub(r"-", r"", filtered)
    filtered = re.sub(r"\d", r"", filtered)
    filtered = re.sub(r"\\", r"", filtered)
    filtered = re.sub(r"\"", r"", filtered)
    filtered = re.sub(r"\s+", r" ", filtered)
    filtered = filtered.lower()
    cleaned_sentence_list.append(f"sos {filtered}")
  return cleaned_sentence_list



In [4]:
def clean_book_and_tokenize_sentences(book):
  '''
  '''
  list_of_undesirables = ["STAVE I:  MARLEY'S GHOST", "STAVE II:  THE FIRST OF THE THREE SPIRITS", "STAVE III:  THE SECOND OF THE THREE SPIRITS",
                   "A CHRISTMAS CAROL", "Stave IV:  The Last of the Spirits", "Stave V:  The End of It"]
  for string in list_of_undesirables:
    book = re.sub(string, r"", book)
  book = re.sub("\n", r" ", book)
  sentence_list = [". marley was dead."]
  appendage = re.findall(r"\..*?\.", book)
  for appendend in appendage:
    sentence_list.append(appendend)
  for i, sentence in enumerate(sentence_list):
    sentence_list[i] = sentence[1:].strip()
  cleaned_sentence_list = clean_sentences(sentence_list)
  return cleaned_sentence_list


In [5]:
sentence_list = clean_book_and_tokenize_sentences(scroodge_book)

In [6]:
a = [1,2,3, 4, 5, 6]
b = [2,3,4,5,6]
c = [3,4,5,6]
d = zip(a,b,c)
for i in d:
  print(i)

(1, 2, 3)
(2, 3, 4)
(3, 4, 5)
(4, 5, 6)


In [7]:
sentence_list[-20:]

['sos your uncle scrooge eos',
 "sos will you let me in fred eosq let him in it is a mercy he didn't shake his arm off eos",
 'sos nothing could be heartier eos',
 'sos so did topper when he came eos',
 'sos so did every one when they came eos',
 'sos oh he was early there eos',
 'sos and he did it yes he did the clock struck nine eos',
 'sos a quarter past eos',
 'sos he was full eighteen minutes and a half behind his time eos',
 'sos his hat was off before he opened the door his comforter too eos',
 'sos hallo growled scrooge in his accustomed voice as near as he could feign it eos',
 'sos i am behind my time eos',
 'sos yes eos',
 'sos step this way sir if you please eos',
 'sos it shall not be repeated eos',
 "sos  now i'll tell you what my friend said scrooge i am not going to stand this sort of thing any longer eos",
 'sos he had a momentary idea of knocking scrooge down with it holding him and calling to the people in the court for help and a straitwaistcoat eos',
 "sos a merrie

In [8]:
def generate_one_hot_mapping(sentence_list):
  '''
  '''
  words_to_int = {}
  int_to_words = {}
  size = 0
  for sentence in sentence_list:
    words = re.split(r"\s", sentence)
    for word in words:
      if word in words_to_int.keys():
        continue
      else:
        words_to_int[word] = size
        int_to_words[size] = word
        size += 1
  return (int_to_words, words_to_int)


In [9]:
def translate_to_num(sentence_list):
  '''
  '''
  num_dict, word_dict = generate_one_hot_mapping(sentence_list)
  translated_sequences = []
  for sentence in sentence_list:
    words = re.split(r"\s", sentence)
    sequence = []
    for word in words:
      try:
        sequence.append(word_dict[word])
      except Exception:
        continue
    translated_sequences.append(sequence)
  return translated_sequences, num_dict, word_dict

In [64]:
def generate_training_data(sentence_list):
  '''
  '''
  tensor_list = []
  string_sequences, num_dict, word_dict = translate_to_num(sentence_list)
  word_dimension = len(num_dict.keys())
  for sentence in string_sequences:
    if len(sentence) >= 7:
      zipped = zip(sentence, sentence[1:], sentence[2:], sentence[3:], sentence[4:], sentence[5:], sentence[6:])
      for tup in zipped:
        if tup in tensor_list:
          continue
        tensor_list.append(tup)
  matrix_list = []
  for tup in tensor_list:
    tensor = torch.zeros((7, word_dimension))
    for i in range(len(tup)):
      tensor[i,tup[i]] = 1
    matrix_list.append(tensor)
  return matrix_list, num_dict, word_dict

In [65]:
string_matrices, num_dict, word_dict = generate_training_data(sentence_list)

In [12]:
string_matrices[:5]

[tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]])]

In [66]:
def to_strings(string_matrices, num_dict):
  '''
  '''
  string_list = []
  for matrix in string_matrices:
    word1 = num_dict[torch.argmax(matrix[0,:]).item()]
    word2 = num_dict[torch.argmax(matrix[1,:]).item()]
    word3 = num_dict[torch.argmax(matrix[2,:]).item()]
    word4 = num_dict[torch.argmax(matrix[3:]).item()]
    word5 = num_dict[torch.argmax(matrix[4:]).item()]
    word6 = num_dict[torch.argmax(matrix[5:]).item()]
    word7 = num_dict[torch.argmax(matrix[6:]).item()]
    string_list.append((word1,word2,word3, word4, word5, word6, word7))
  return string_list

In [67]:
string_list = to_strings(string_matrices, num_dict)

In [68]:
string_list[:30]

[('sos', 'there', 'is', 'no', 'doubt', 'whatever', 'about'),
 ('there', 'is', 'no', 'doubt', 'whatever', 'about', 'that'),
 ('is', 'no', 'doubt', 'whatever', 'about', 'that', 'eos'),
 ('sos', 'scrooge', 'signed', 'it', 'and', "scrooge's", 'name'),
 ('scrooge', 'signed', 'it', 'and', "scrooge's", 'name', 'was'),
 ('signed', 'it', 'and', "scrooge's", 'name', 'was', 'good'),
 ('it', 'and', "scrooge's", 'name', 'was', 'good', 'upon'),
 ('and', "scrooge's", 'name', 'was', 'good', 'upon', "'change"),
 ("scrooge's", 'name', 'was', 'good', 'upon', "'change", 'for'),
 ('name', 'was', 'good', 'upon', "'change", 'for', 'anything'),
 ('was', 'good', 'upon', "'change", 'for', 'anything', 'he'),
 ('good', 'upon', "'change", 'for', 'anything', 'he', 'chose'),
 ('upon', "'change", 'for', 'anything', 'he', 'chose', 'to'),
 ("'change", 'for', 'anything', 'he', 'chose', 'to', 'put'),
 ('for', 'anything', 'he', 'chose', 'to', 'put', 'his'),
 ('anything', 'he', 'chose', 'to', 'put', 'his', 'hand'),
 ('he',

In [69]:
class RNN(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    output_matrix = torch.rand((output_size, hidden_size), requires_grad=True)
    self.output_layer = nn.Parameter(output_matrix, requires_grad=True)
    self.output_layer_bias = nn.Parameter(torch.rand((output_size)), requires_grad=True)
    self.loss = nn.CrossEntropyLoss()
    self.output_size = output_size
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.hidden_layer = nn.RNNCell(input_size, hidden_size)

  def forward(self, input_batch):
    hidden_list = []
    total_time = input_batch.shape[0]
    size_of_batch = input_batch.shape[1]
    h0 = torch.zeros((size_of_batch, self.hidden_size))
    for time_step in input_batch:
      h0 = self.hidden_layer(time_step, h0)
      hidden_list.append(h0)
    final_hidden = hidden_list[-1]
    output = []
    for tensor in final_hidden:
      current = (self.output_layer @ tensor) + self.output_layer_bias
      output.append(current)
    output = torch.vstack(tuple(output))
    return output



In [79]:
def train(model, string_matrices, epochs):
  '''
  '''
  model.train()
  optim = torch.optim.Adam(model.parameters())
  batched = batch_matrices(string_matrices, 7)
  for i, batch in enumerate(batched):
    training = batch[:7]
    target = batch[-1]
    print(f"batch: {i}")
    avg_loss = float('inf')
    maximum_iterations = 1
    last_improved = 0
    accuracy = 0
    while accuracy < 90 and maximum_iterations < epochs and last_improved < 4:
      print(f"epoch: {maximum_iterations}")
      optim.zero_grad()
      output = model.forward(training)
      loss = model.loss(output, target)
      loss.backward()
      optim.step()
      avg_loss = loss.item()
      maximum_iterations+=1
      current_accuracy = model_accuracy(model, training, target)
      print(f"batch accuracy: {current_accuracy}")
      if current_accuracy <= accuracy:
        last_improved+=1
      accuracy = current_accuracy


In [18]:
def batch_matrices(string_matrices, n):
  '''
  '''
  N = 500
  length = len(string_matrices)
  batches = int(length / N)
  remainder = length % N
  batched_matrices = []
  for i in range(batches):
    batch = string_matrices[i * N: (i + 1) * N]
    input_tensor = []
    for t in range(n):
      current_time = []
      for matrix in batch:
        current_time.append(matrix[t,:])
      input_tensor.append(torch.vstack(tuple(current_time)))
    input_tensor = torch.vstack(tuple(input_tensor))
    input_tensor = torch.reshape(input_tensor, (n, N, -1))
    batched_matrices.append(input_tensor)
  remaining_batch = string_matrices[length - remainder: length]
  remainder_tensor = []
  for t in range(n):
    current_time = []
    for matrix in remaining_batch:
      current_time.append(matrix[t,:])
    current_time = torch.vstack(tuple(current_time))
    remainder_tensor.append(current_time)
  remainder_tensor = torch.vstack(tuple(remainder_tensor))
  remainder_tensor = torch.reshape(remainder_tensor, (n, remainder, -1))
  batched_matrices.append(remainder_tensor)
  return batched_matrices


In [19]:
def model_accuracy(model, input_data, target_data):
  '''
  '''
  model.eval()
  softmax = nn.Softmax(dim=1)
  predictions = torch.argmax(softmax(model.forward(input_data)), dim=1)
  target = torch.argmax(target_data, dim=1)
  score = 100 * (torch.sum(torch.eq(predictions, target))).item() / len(target_data)
  return score


In [72]:
vocab_size = len(num_dict.keys())
print(f"vocab size: {vocab_size}", f"training_size: {len(string_matrices)}")
input_size = vocab_size
hidden_size = 1000
output_size = vocab_size
total_params = hidden_size * input_size + hidden_size * hidden_size + output_size * hidden_size + hidden_size + hidden_size + output_size
print(f"total parameters: {total_params}", f"\napproximate size: {float((total_params * 32) / float(10**9))} giga bytes")

vocab size: 2917 training_size: 11462
total parameters: 6838917 
approximate size: 0.218845344 giga bytes


In [76]:
rnn = RNN(input_size, hidden_size, output_size)

In [80]:
train(rnn, string_matrices, 15)

batch: 0
epoch: 1
batch accuracy: 10.2
epoch: 2
batch accuracy: 9.6
epoch: 3
batch accuracy: 36.2
epoch: 4
batch accuracy: 84.6
epoch: 5
batch accuracy: 94.8
batch: 1
epoch: 1
batch accuracy: 50.0
epoch: 2
batch accuracy: 51.6
epoch: 3
batch accuracy: 52.6
epoch: 4
batch accuracy: 54.4
epoch: 5
batch accuracy: 58.2
epoch: 6
batch accuracy: 54.2
epoch: 7
batch accuracy: 54.6
epoch: 8
batch accuracy: 57.2
epoch: 9
batch accuracy: 52.2
epoch: 10
batch accuracy: 51.0
epoch: 11
batch accuracy: 54.4
epoch: 12
batch accuracy: 54.0
batch: 2
epoch: 1
batch accuracy: 32.0
epoch: 2
batch accuracy: 34.8
epoch: 3
batch accuracy: 31.2
epoch: 4
batch accuracy: 42.2
epoch: 5
batch accuracy: 55.2
epoch: 6
batch accuracy: 58.0
epoch: 7
batch accuracy: 59.2
epoch: 8
batch accuracy: 58.0
epoch: 9
batch accuracy: 58.8
epoch: 10
batch accuracy: 62.0
epoch: 11
batch accuracy: 63.0
epoch: 12
batch accuracy: 63.0
epoch: 13
batch accuracy: 62.4
batch: 3
epoch: 1
batch accuracy: 56.0
epoch: 2
batch accuracy: 59.

In [25]:
def predict_next_word(model, sentence, num_dict, word_dict):
  '''
  '''
  filtered = re.sub(r"[;:,]", r"", sentence)
  filtered = re.sub(r"\.", r" eos", filtered)
  filtered = re.sub(r"\?", r" eosq", filtered)
  filtered = re.sub(r"!", r"", filtered)
  filtered = re.sub(r"-", r"", filtered)
  filtered = re.sub(r"\d", r"", filtered)
  filtered = re.sub(r"\\", r"", filtered)
  filtered = re.sub(r"\"", r"", filtered)
  filtered = re.sub(r"\s+", r" ", filtered)
  filtered = filtered.lower()
  token_list = re.split(r"\s", filtered)
  token_list = token_list[-3:]
  tensor = []
  for token in token_list:
    d = word_dict[token]
    t = torch.zeros((len(num_dict.keys()),))
    t[d] = 1
    tensor.append(t)
  tensor = torch.vstack(tuple(tensor))
  tensor = torch.reshape(tensor, (3, 1, len(num_dict.keys())))
  output = model.forward(tensor)
  output = output[0]
  prediction = torch.argmax(output).item()
  return num_dict[prediction]

In [126]:
print(sentence_list[18])

sos once upon a timeof all the good days in the year on christmas eveold scrooge sat busy in his countinghouse eos


In [127]:
generate_text(rnn, "once upon a timeof all the ", num_dict, word_dict)

ceiling
and
known
of
the
goose
and
known
of
the
goose


In [35]:
def generate_text(model, string, num_dict, word_dict):
  '''
  '''
  token = predict_next_word(model, string, num_dict, word_dict)
  n = 0
  while(token != 'eos' and n < 10):
    print(token)
    token = predict_next_word(model, f"{string} {token} ", num_dict, word_dict)
    n+=1
  print(token)